# **Chapter 3: Code Generation**

---
**Lesson:**

The ability to generate code using a Large Language Model (LLM) is a powerful feature. Boiler plate or even custom feature code, while not challenging to create, can be tedious and time-consuming. 

Code generation can help speed up feature and functionality development by offloading the development work to an LLM, like Mistral 7B. 

Below, we'll walk step by step through on how to create a prompt for Mistral 7B to generate code for us.

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3
import botocore.exceptions

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

# boto3_bedrock = bedrock.get_bedrock_client(
#     assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
#     region=os.environ.get("AWS_DEFAULT_REGION", None)
# )

session = boto3.Session(
    profile_name="default"
) #sets the profile name to use for AWS credentials

boto3_bedrock = session.client(
    service_name='bedrock-runtime', #creates a Bedrock client
    region_name="us-west-2"
) 

modelId = "mistral.mistral-7b-instruct-v0:2" # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_tokens': 400,
        'top_p': 0.7,
        'top_k': 50,
        'temperature': 0.7
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputs = response_body["outputs"]
        outputText = [output["text"] for output in outputs]
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Examples:**

Let's take a look at how Mistral 7B responds to some basic code generation tasks. Don't worry about how the answers are being generated for now. Please run the example cells below to get the response from Mistral 7B.

**Example 3.1 - Basic Python function to convert Celsius to Fahrenheit**

In [ ]:
# create the prompt
prompt_data = """<s>[INST]Create a Python function to convert Celsius to Fahrenheit.[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 3.2 - Basic Bash script to delete all files in a directory**

**NOTE)** Amazon Bedrock catches the potential harm of the script generated by this prompt.

In [ ]:
# create the prompt
prompt_data = """<s>[INST]Create a Bash script that will delete all files in a directory I pass in as an argument.[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 3.3 - Using a system prompt to help define rules and instructions for code generation**

In [ ]:
# create the prompt
prompt_data = """<s>[INST]You are a coding assistant that can only write Python code. If asked for any other language besides Python, please respond with 'I can only generate Python code'.
Create a JavaScript function returns the latest stock price for Amazon.[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# **Exercises**

The following two exercises will need you to manipulate the prompt to get the desired output

**Exercise 3.1 - No explanation, just code**

Using a system prompt, update the value **REPLACE_ME_WITH_PROMPT** in the cell below to get Mistral 7B to only generate code with no comments or explanation.

In [ ]:
# create the prompt
prompt_data = """<s>[INST]REPLACE_ME_WITH_PROMPT.
Create a unit test in Python using the package unittest from the following function
def add(a, b):
    return a + b[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Exercise 3.2 - Translate code**

This notebook mainly focuses on code generation, but did you know you can convert code from one langage to the next using an LLM? Update the following prompt so Mistral 7B converts a Python function to a language of your choosing. Make sure to update the value **REPLACE_ME_WITH_LANGUAGE** in the cell below.

In [ ]:
# create the prompt
prompt_data = """<s>[INST]Convert the below Python function to REPLACE_ME_WITH_LANGUAGE.
def add(a, b):
    return a + b[/INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# Chapter 3 - END.